In [ ]:
%pip install -q 

| Area                                | Points |
| ----------------------------------- | ------ |
| Scrape & clean quality              | 25     |
| API-based vector-store upload works | 20     |
| Daily job deployment & logs         | 15     |
| Code clarity + README               | 10     |
| Bonus (tests, improvements)         | +5     |
